In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Data processing

Load the daily prompt survey into pandas. Group by patients and label each generated 7-day data window according to whether they contain an amber-zone. All 7 days in each window are labelled the same.

In [ ]:
daily = pd.read_csv('data/Daily prompt survey.csv').drop(['ROW_ID','ROW_VERSION','recordId','appVersion','phoneInfo'],axis=1)
daily['get_worse_counts']=daily['get_worse'].str.count('\d+')   # sum total number of asthma triggers
daily['unix_time'] = daily['createdOn'] # convert unix time to date format
daily['createdOn'] = pd.to_datetime(daily['createdOn'], unit='ms').dt.normalize()   # normalise date format

warnings.filterwarnings('ignore')
data = list()

# group by patient (healthCode)
for key, grp in daily.groupby(['healthCode']):
    if len(grp)>49:     # select patients with 50+ entries

        # split patient data timeframe into windows of 7 days and bin patient data entries accordingly
        interval_range = pd.interval_range(start=grp['createdOn'].iloc[0], freq='7D', end=grp['createdOn'].iloc[-1]+pd.DateOffset(7), closed='left')
        grp['bin'] = pd.cut(grp['createdOn'], bins=interval_range)

        pfmax = np.nanmax(grp['peakflow'])  # used for peak flow normalisation
        
        # group all data entries per window (still per patient)
        for x, y in grp.groupby(['bin']):

            # label amber-zones using criteria below
            night = y['night_symptoms'].any()
            qr = (y['quick_relief_puffs']>=3).any()
            y['PEF'] = y['peakflow']/pfmax
            pf = (y['peakflow']<pfmax*0.7).any()
            
            label = 0
            if night and pf:
                label=1
            if qr:
                label=1
            
            y['label']=label

            data.append(y)
daily_label = pd.concat(data, ignore_index=True)

daily_label.loc[daily_label['use_qr']==False, 'quick_relief_puffs']=0



daily_label.head()

## Feature Extraction

With the data labelled, feature extraction is next. This involves making a feature window 7 days before the labelled window until the start of the labelled window. Features are extracted from this new window.

In [ ]:
data = list()
linreg = LinearRegression()    # define model for finding gradients later

# group by patient
for key, grp in daily_label.groupby(['healthCode']):

        # group by labelled window
        for x,y in grp.groupby(['bin']):
            start = x.left-pd.DateOffset(7) # find date 7 days before labelled window
            end = x.left    # find date at start of labelled window
            label = y['label'].mean()   # find label
            intermed = grp[(grp.createdOn < end) & (start <= grp.createdOn)]    # define feature window using start and end dates and capture all relevant data entries

            # filter feature windows by number of entries. All fields must contain >2 entries
            if len(intermed)>1 and intermed['quick_relief_puffs'].count()>2 and intermed['day_symptoms'].count()>2 and intermed['night_symptoms'].count()>2 and intermed['get_worse_counts'].count()>2 and intermed['peakflow'].count()>2:
                
                # create mask to filter out NaN values as linreg cannot handle these. Apply mask to all feature fields
                mask = ~np.isnan(intermed['unix_time']) & ~np.isnan(intermed['quick_relief_puffs']) & ~np.isnan(intermed['get_worse_counts']) & ~np.isnan(intermed['PEF'])
                unix = (intermed['unix_time']/10**10)[mask].values.reshape(-1,1)
                qr = intermed['quick_relief_puffs'][mask].values.reshape(-1,1)
                worse = intermed['get_worse_counts'][mask].values.reshape(-1,1)
                pef = intermed['PEF'][mask].values.reshape(-1,1)

                # extract features from fields. Use linreg for finding gradient
                qr_mean = intermed['quick_relief_puffs'].mean()
                qr_grad = linreg.fit(unix,qr).coef_[0][0]   # returns gradient
                qr_abs = abs(qr_grad)
                

                day_mean = (intermed['day_symptoms']==True).mean()  # sum number of entries with day symptoms = true
                night_mean = (intermed['night_symptoms']==True).mean()


                worse_mean = intermed['get_worse_counts'].mean()
                worse_grad = linreg.fit(unix,worse).coef_[0][0]
                worse_abs = abs(worse_grad)

                
                pef_mean = intermed['PEF'].mean()
                pef_grad = linreg.fit(unix,pef).coef_[0][0]
                pef_abs = abs(pef_grad)

                total = len(intermed)   # total number of entries in feature window

                data.append((key,x,qr_mean,qr_grad,qr_abs,day_mean,night_mean,worse_mean,worse_grad,worse_abs,pef_mean,pef_grad,pef_abs,total,label))
            else:
                pass

# place results into new dataframe
daily_features = pd.DataFrame(data)
daily_features.columns = ['user','date range','qr_mean','qr_grad','qr_abs','day_mean','night_mean','worse_mean','worse_grad','worse_abs','pef_mean','pef_grad','pef_abs','total','label']
print(daily_features.shape)
print(daily_features['label'].sum())
daily_features


## Feature Selection

Here, the features are ranked on their importance to prediction. A forward feature selector using a logistic regression classifier and a maximum relevance-minimum redundancy algorithm are used. MRMR is used in the report.

In [ ]:
from sklearn.model_selection import train_test_split

# select columns containing the features and split into a test and validation selection
feature_cols = ['qr_mean','qr_grad','qr_abs','day_mean','night_mean','worse_mean','worse_grad','worse_abs','pef_mean','pef_grad','pef_abs','total']

X = daily_features.loc[:,feature_cols]
y = daily_features.label

# split into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

Forward feature selection:

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

# select model for sfs
logreg = LogisticRegression()

# define sequential feature selector (sfs)
sfs = SFS(logreg,
          k_features=11, # want to rank all features ( k = max total features -1)
          forward=True, # want selection direction to be forward (add features)
          floating=False,
          scoring = 'f1',
          cv = 0)

# fit sfs model to data
sfs.fit(X_train, y_train, custom_feature_names=feature_cols)

# retrieve feature scores
features_df=pd.DataFrame.from_dict(sfs.get_metric_dict()).T

features_df

MRMR:

In [ ]:
from sklearn.feature_selection import f_regression

# inputs:
#    X: pandas.DataFrame, features
#    y: pandas.Series, target variable
#    K: number of features to select

# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, y)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()
score_list = []

# repeat K times
for i in range(10):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)
    score_list.append(score[score.argmax()])

print(selected)
print(score_list)
F


# Models

Here, 5 different classifiers are tested on the data. GridsearchCV is used to find the best hyperparameters for the models. Repeated CV is then used to evaluate the models. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn import model_selection

feature_cols = ['qr_mean','day_mean','night_mean','worse_mean','pef_mean','total']  # select features to be taken forward

X = daily_features.loc[:,feature_cols]
y = daily_features.label

## GridSearch

Parameters to be tested are defined per classifier. Trialling is repeated 5 times. The scoring method used is f1.

### SVM

In [ ]:
parameters = {'C':range(1,5,1),'gamma':range(1,5,1)}    # define params

grid = model_selection.GridSearchCV(SVC(),parameters, scoring = 'f1',cv=5)

grid.fit(X,y)

grid_svm = pd.DataFrame(grid.cv_results_)

grid.best_params_

### RF

In [ ]:
parameters = {'n_estimators':range(10,160,10),'max_features':['sqrt','log2',6]}
grid = model_selection.GridSearchCV(RandomForestClassifier(),parameters, scoring = 'f1',cv=5)

grid.fit(X,y)

grid_rf = pd.DataFrame(grid.cv_results_)

grid.best_params_

### kNN

In [ ]:
parameters = {'n_neighbors':range(1,10),'metric':['euclidean', 'manhattan', 'minkowski'],'p':[1,2]}
grid = model_selection.GridSearchCV(KNeighborsClassifier(),parameters, scoring = 'f1',cv=5)

grid.fit(X,y)

grid_knn = pd.DataFrame(grid.cv_results_)

grid.best_params_

## Final

Once the optimal hyperparameters are found, the models are tested. Here 5x5 CVs are used to validate each model. Results are tabulated and displayed as boxplots. 6 evaluation metrics are used. ROCs are also calulated.

In [ ]:
# define models with relevant hyperparameters
models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier(n_estimators=130, max_features='log2')),
          ('kNN', KNeighborsClassifier(n_neighbors=9,metric='euclidean',p=1)),
          ('SVM', SVC(C=1,gamma=1)), 
          ('GNB', GaussianNB())
        ]

# initiate trackers
names = []
dfs = []
dfs1 = []

# define scoring/evaluation metrics
scoring = {'accuracy':'accuracy', 
          'precision':'precision', 
          'recall':'recall', 
          'specificity': make_scorer(recall_score,pos_label=0), # specificity scorer has to to seperately defined as it is not standard
          'f1':'f1', 
          'roc_auc':'roc_auc'
          }

# perform repeated CV on models
for name, model in models:
    kfold = model_selection.RepeatedKFold(n_splits=5, n_repeats=5)  # define CV folds
    cv_results = model_selection.cross_validate(model, X, y, cv=kfold, scoring=scoring) # perform CV and return results

    # place results temporarily into pandas df as we go along and calculate median
    names.append(name)
    df = pd.DataFrame(cv_results)
    median = pd.DataFrame(df.median().to_dict(),index=[df.index.values[-1]])
    df['model']=name
    median['model']=name
    dfs.append(df)
    dfs1.append(median)

# concat median results
final = pd.concat(dfs, ignore_index=True)
final_median = pd.concat(dfs1, ignore_index=True)
final_median


create boxplots

In [ ]:
import seaborn as sns
sns.set_style("white")

# define figure properties
rc = {'axes.facecolor':'white',
      'axes.grid' : True,
      'grid.color': '.8',
      'font.family':'Times New Roman',
      'font.size' : 15}
plt.rcParams.update(rc)

# process data so seaborn can use it
final_long = final.drop(['fit_time','score_time'],axis=1)
final_long.columns = ['Accuracy','Precision','Recall/Sensitivity','Specificity','F1-score','AUC','Model']
df_long = pd.melt(final_long,'Model', var_name='Evaluation Metric', value_name='Score')   # melting stage is important

# plot data
sns.catplot('Evaluation Metric', hue='Model', y='Score', data=df_long, kind='box', height=5, aspect=2.5)
sns.despine(fig=None, ax=None, top=False, right=False, left=False, bottom=False, offset=None, trim=False)

Calculate mean ROC curves for each model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

colour = ['b','r','y','g','m']
j=0

# define CV splits
cv1 = StratifiedKFold(n_splits=5, shuffle=True)

# convert to numpy
X_num = X.to_numpy()
y_num = y.to_numpy()

fig1, ax1 = plt.subplots()
fig, ax = plt.subplots(figsize=(9,7))

for name, model in models:

    # initiate trackers
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    # perform CV on models, but this time create ROC curves
    for i, (train, test) in enumerate(cv1.split(X_num, y_num)):
        model.fit(X_num[train], y_num[train])
        viz = RocCurveDisplay.from_estimator(
            model,
            X_num[test],
            y_num[test],
            name=None,
            alpha=0,
            lw=1,
            ax=ax1,     # place each individual CV result onto throw-away figure (simple workaround having all results on one plot)
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    
    # calculate mean ROC
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    # plot mean ROCs on actual plot
    ax.plot(
        mean_fpr,
        mean_tpr,
        label="{}".format(name)+r' (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc,std_auc),
        color=colour[j],
        lw=2,
        alpha=0.8,
    )
    j+=1

# style plot
ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="k", label="Chance", alpha=0.8)    
ax.legend(loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')


plt.show()